<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLMEVALUATOR_OPENAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install lm-eval -q

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q

In [4]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
openai.api_key = os.environ["OPENAI_API_KEY"]
#print(openai.api_key)

In [ ]:
#harness_repo="public-lm-eval-harness"

!git clone https://github.com/EleutherAI/lm-evaluation-harness/
%cd /content/lm-evaluation-harness

!pip install -q -e .

In [ ]:
!pip install datasets@git+https://github.com/huggingface/datasets.git@66d6242 -q
!pip install tokenizers>=0.15.2 transformers>=4.38.2 sentencepiece>=0.2.0 -q

In [ ]:
!which lm_eval

https://platform.openai.com/docs/guides/text-generation

In [9]:
%mkdir -p /content/lm_eval_output
%rm -rf /content/lm_eval_output/*

https://platform.openai.com/docs/api-reference/models

In [8]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'),
 Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
 Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
 Model(i

In [9]:
modelid=modellist.data[4].id
modelid

'gpt-4o'

/content/lm-evaluation-harness/lm_eval/models/openai_completions.py -- line=474


In [13]:
import random
def loglikelihood(self, requests, disable_tqdm: bool = False):
        res = []

        for _ in tqdm(requests, disable=disable_tqdm):
            res.append((-random.random(), False))

        return res

In [15]:
from openai import OpenAI

system_content = "You are a travel agent. Be descriptive and helpful."
user_content = "Tell me about havana"


client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

chat_completion = client.chat.completions.create(
    model=modelid,
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=0.9,
    max_tokens=1024,
    #echo=True
)

response = chat_completion.choices[0].message.content
print("Together response:\n", response)

#TypeError: Completions.create() got an unexpected keyword argument 'echo'

Together response:
 Absolutely, I'd be delighted to tell you about Havana!

Havana, the capital city of Cuba, is an enchanting destination brimming with history, culture, and a vibrant rhythm that captures the hearts of its visitors. Here's a comprehensive look at what makes Havana so special:

### Historical Significance
Havana is steeped in history, with its roots stretching back to its founding in 1519 by Spanish colonists. The city's rich past is vividly brought to life through its architecture, museums, and historical sites.

### Architectural Marvels
One of the first things you'll notice in Havana is its stunning architecture. The city boasts a mix of colonial, baroque, neoclassical, and Art Deco buildings. Notable sites include:

- **Old Havana (Habana Vieja)**: A UNESCO World Heritage site, this area is a treasure trove of colonial architecture with cobblestone streets, historic plazas, and beautifully preserved buildings. Key landmarks include the Plaza de la Catedral, the Cas

https://github.com/EleutherAI/lm-evaluation-harness/

In [24]:
import os
#modelid='gpt-4'
#modelid='gpt-3.5-turbo-instruct'

os.environ['model'] = modelid
#os.environ['task']='mmlu'

os.environ['task']='lambada_openai,hellaswag'
#os.environ['task']='mmlu,lambada_openai,openbookqa,arc_easy,winogrande,hellaswag,arc_challenge,piqa,boolq'
os.environ['shot']='0'
os.environ['batch_size']='1'

os.environ['tokenizer']= modelid
os.environ['add_bos_token']='True'

os.environ['log_samples']='True'
os.environ['random_seed']='0'
os.environ['trust_remote_code']='True'

In [15]:
#!lm_eval --model hf --model_args pretrained=EleutherAI/gpt-j-6b,parallelize=True,load_in_4bit=True,peft=nomic-ai/gpt4all-j-lora --tasks openbookqa,arc_easy,winogrande,hellaswag,arc_challenge,piqa,boolq --device cuda:0

In [25]:
!echo $model
!echo $task
!echo $shot
!echo $batch_size
#print(os.environ)
print()

# Supported model names: anthropic, anthropic-chat, anthropic-chat-completions, dummy, gguf, ggml, hf-auto, hf, huggingface, mamba_ssm, nemo_lm, sparseml, deepsparse, neuronx, openai-completions, local-completions, openai-chat-completions, local-chat-completions, openvino, textsynth, vllm
# trust_remote_code=True,
!lm_eval --model openai-chat-completions --model_args model=${model},tokenizer=${tokenizer}   --tasks ${task}  --num_fewshot ${shot} --output_path /content/lm_eval_output/${model//\//_}_${task//,/_}-${shot}shot --batch_size ${batch_size} 2>&1 | tee /content/lm_eval_output/eval-${model//\//_}_${task//,/_}-${shot}shot.log
#!lm_eval --model openai-completions --verbosity DEBUG --model_args model=${model},tokenizer=${tokenizer},random_seed=${random_seed},log_samples=${log_samples},trust_remote_code=${trust_remote_code}   --tasks ${task}  --num_fewshot ${shot} --output_path /content/lm_eval_output/${model//\//_}_${task//,/_}-${shot}shot --batch_size ${batch_size} 2>&1 | tee /content/lm_eval_output/eval-${model//\//_}_${task//,/_}-${shot}shot.log

gpt-4o
lambada_openai,hellaswag
0
1

2024-05-15 00:22:49.299746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 00:22:49.299807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 00:22:49.301305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 00:22:50.731273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-15:00:22:54,933 INFO     [__main__.py:254] Verbosity set to INFO
2024-05-15:00:23:02,025 INFO     [__main__.py:341] Selected Tasks: ['hellaswag', 'lambada_openai']
2024-05-15:00:23:02,027 INFO    